Train - CV - Test = 60:20:20

In [1]:
# IMPORT MODULES

%matplotlib inline

import datetime
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dts
import calendar

import sklearn.preprocessing as pp
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold as KF
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as rms

import pandas as pd

In [2]:
# READ DATA
data = pd.read_csv("DonneesMeteo.txt", sep=" ", header=1)
data["Date"] = pd.to_datetime(data["Date"], format="%Y%m%d")

data['MONTH'] = date.month
data['DAY'] = date.day
data['YEAR'] = date.year

hh = pd.DatetimeIndex(pd.to_datetime(df['hh(UTC)']*3600*1000000000))
data['HOUR'] = hh.hour
data['MINUTE'] = hh.minute
data

In [3]:
dataset = np.array(data[["hh(UTC)", "IrrPOA(W/m2)", "P1(W)"]]).astype(np.float)

#Remove nan and infinite values
masknan = ~np.any(np.isnan(dataset), axis=1)
dataset = dataset[masknan]
maskfin = np.any(np.isfinite(dataset), axis=1)
dataset = dataset[maskfin]
X=dataset[:,0:2]
y=dataset[:,2]

X[:,1]=np.round(X[:,1],decimals=0)

#LABEL BINARIZER : editting time data
lb = pp.LabelBinarizer()
lb.fit([1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19, 20, 21, 22, 23, 0])
pp.LabelBinarizer(neg_label=0, pos_label=1)
lb.classes_
Hour=lb.fit_transform(X[:,1])
Irr = np.reshape(X[:,0],(len(X),1))

X=np.concatenate((Irr,Hour),axis=1)

In [4]:
#Split Training + CV and Test set
X_training, X_test, y_training, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#SCALE INDEPENDENT DATA
scaler = pp.StandardScaler()
X_training=scaler.fit_transform(X_training)
X_test=scaler.transform(X_test)

#Parameters to optimize
parameters = [{'kernel': ['rbf'], 'gamma': [1, 1e-1, 1e-2, 1e-3, 1e-4], 'C': [1, 10, 100]}]

In [6]:
scores = {r2_score, rms}
for score in scores:
    clf = GridSearchCV(SVR(C=1), parameters, scoring=score, cv=8)
    clf.fit(X_training, y_training)
    
    print("Best parameters set found on development set using: ")
    print(clf.best_params_)
    
    y_true, y_pred = y_test, clf.predict(X_test)
    
    print(score)
    print(score(y_true, y_pred))

TypeError: Expected sequence or array-like, got estimator SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)